# Numerical Methods Project 2 by Sercan Hüsnügil

## Question 1

1. As demonstrated in the lecture, approximate fstar(x) = sin(x) on the interval [0, pi/2] via polynomials using a least squares approach. Use 1,000 evaluation points. Create a repository on Github holding your code. This repository needs to have a README explaining (in brief) the algorithm, as well as instructions for using your code to reproduce the results. These instructions should also include figures since a picture is worth a thousand words. Your code should be able to show results for all tasks in this project, and your code should be well-written and understandable. Use comments.

We want to approximate $f^*(x) = \sin{(x)}$ in $[0, \pi/2]$ via a least squares method.

Recall that we formulated the least squares method in the lectures as the following:

Let $f^*(x)$ be the function we try to approximate. We can expand the function as the sum of polynomials up to an order $p$:
$f^*(x) = \sum_n^p c_n x^n$ where $x^n$ is defined in an interval with $R$ number of points.

Least squares method aims to minimize the error between the actual and approximated values: 

\begin{equation}
E = \sum_i^R (f^*(x_i) - f(x_i))^2 
\end{equation}

In [1]:
using WGLMakie # Used for plots.

In [2]:
# Define the function to calculate the approximate values.

function least_squares(pmax, npoints, x, fstar, xx)
    """
    Returns f(x) values which is an approximation of fstar(x) using least squares method. Values are evaluated over the x-axis xx.
    
    pmax: order of expansion
    npoints: number of evaluation points
    x: range of x values e.g. LinRange(0, π/2, npoints)
    fstar(x): function to be approximated
    xx: x axis of the plot e.g. LinRange(0, π/2, 201)
    
    """
    
    
    A = [x[i]^p for i in 1:npoints, p in 0:pmax]; # Matrix of c_p*(x_i)^p values

    b = fstar.(x); # Values of fstar on range of x

    c = (A'*A) \ (A'*b)  # Vector of coefficients
    
    f(c,x) = sum(c[p+1] * x^p for p in 0:pmax); # Approximate function
    
    return yy = [f(c,x) for x in xx] 
end

least_squares (generic function with 1 method)

In [3]:
# Parameters to plug in least_squares(pmax, npoints, x, fstar, xx)
pmax = 5
npoints = 1000
x = LinRange(0, π/2, npoints)
xx = LinRange(0, π/2, 201) 
fstar(x) = sin(x)
yy = least_squares(pmax, npoints, x, fstar, xx)

# Plot the approximation and the actual function to campare.
fig = Figure()
ax = Axis(fig[1,1], xlabel=L"x", ylabel=L"f(x)", title = L"$f(x)$ and $f^*(x) = \sin(x)$" )
lines!(xx, fstar.(xx), color=:red, linestyle=:solid, label="fstar(x)") # actual function
scatter!(xx, yy, markersize=3, color=:blue, label="f(x)") # approximated values 
fig[1,2] = Legend(fig,ax)
fig

As it can be seen from the plot, the approximated values via least squares method match well with the target function $f^*(x)=\sin(x)$ for the interval $[0,\pi/2]$.

We can test this method on a larger interval and see if something changes.



In [4]:
# Trying a different interval
x = LinRange(0, π*4, npoints)
xx = LinRange(0, π*4, 201) 

yy = least_squares(pmax, npoints, x, fstar, xx)

# Plot the approximation and the actual function to campare.
fig = Figure()
Axis(fig[1,1], xlabel=L"x", ylabel=L"f(x)", title = L"$f(x)$ and $f^*(x) = \sin(x)$" )
lines!(xx, fstar.(xx), color=:red, linestyle=:solid) # actual function
scatter!(xx, yy, markersize=3, color=:blue) # approximated values 
fig

We see that the current approximation does not perform well for all intervals. One can try to fix this by increasing the maximum polynomial order int the expansion. 

Now, we want to quantify how well this approximation performed.

## Question 2

2. Define the approximation error E as the L2 norm of the difference between the approximation function f(x) and fstar(x). This is also called RMS or "root mean square" of the difference: The square root of the average of (f(x) - fstar(x))^2 at all evaluation points. What polynomial order (pmax) is required to satisfy E < 1.0e-10? Plot the
relation between pmax and E for pmax from 0 to at least 20. Discuss the figure. Can you achieve E < 1.0e-20? Say why.

Following the recipe in the question decription, we define the root mean square error as

\begin{equation}
\text{RMS} = \sqrt{\frac{1}{n}\sum_{i}^{n}(f(x_i) - f^*(x_i))^2}
\end{equation}

In [5]:
# Define a function to calculate the error.
function errorl2(y, ystar)
    """
        Returns root mean square value quantifying the mismatch between ystar and y.
        
        ystar: vector of target (actual) values
        y: vector approximated values
    """
    
    total =  sum((y .- ystar).^2) 
    average = total./size(x,1)
    rms = sqrt(average)
    return rms
    
end;

In [6]:
# Calculating RMS, using the functions least_squares(...) and errorl2(...)
pmax = 5 # order of approximation
npoints = 1000 # number of points in approximation
x = LinRange(0, π/2, npoints) 
xx = LinRange(0, π/2, 201)

fstar(x) = sin(x) # target function
ystar = fstar.(xx); # actual values from the target function

yy = least_squares(pmax, npoints, x, fstar, xx) # calculate the approximated y values using the function defined in Question 1

rms = errorl2(yy, ystar)

2.00742896051948e-6

We see that the root mean square (rms) value for the least squares approximation up to 5th order polynomials and using 1000 points is about $2\times 10^{-6}$. 

Now, we investigate the change in rms value as we increase the polynomial order.

In [20]:
# Searching for the polynomial order (p value) which would yield an rms less than 10e-10

pcap = 50 # maximum polynomial order
npoints = 1000 # number of points in approximation
x = LinRange(0, π/2, npoints)
xx = LinRange(0, π/2, 201)

fstar(x) = sin(x) # target function
ystar = fstar.(xx); # actual values from the target function

rms_vals = zeros(pcap+1) # record the rms values

for p in 0:pcap
    yy = least_squares(p, npoints, x, fstar, xx) # calculate the approximated y values using the function defined in Question 1
    rms_vals[p+1] =  errorl2(yy, ystar);
end


In [21]:
# Plot rms vs polynomial order
fig = Figure()
Axis(fig[1,1], xlabel="Order of maximum polynomial", ylabel=L"$\log_{10}(\text{RMS})$", title = "Root mean square vs maximum polynomial order (logarithmic)  " )
scatter!(0:pcap, log10.(rms_vals)) # approximated values
fig

According to the plot where the $y-$axis values rescaled logarithmically, we can achieve RMS values of order $10^{-10}$ when we go to the polynomial order 10. Above that order, the error value does not decrease but instead fluctuates. This might be due to numerical errors ($\textit{e.g.,}$ floating point errors) introduced along the calculation.

## Question 3

3. The function sin(x) is antisymmetric, i.e. sin(x) = - sin(-x). Modify the method to use only antisymmetric polynomials in your approximation. How does this affect the error? Compare results for the same computational cost, i.e. for the same number of polynomials used (not for the same pmax).

We modify the function in question 1 to only take odd-ordered polynomials into account. 

In [22]:
# Define the function to calculate the approximate values.

function least_squares_odd(pmaxodd, npoints, x, fstar, xx)
    """
    Returns f(x) values which is an approximation of fstar(x) using only odd-ordered polynomials. Values are evaluated over the x-axis xx.
    
    pmaxodd: order of expansion, enter an odd number
    npoints: number of evaluation points
    x: range of x values e.g. LinRange(0, π/2, npoints)
    fstar(x): function to be approximated
    xx: x axis of the plot e.g. LinRange(0, π/2, 201)
    
    """
    
    
    A = [x[i]^p for i in 1:npoints, p in 1:2:pmaxodd]; # Matrix of c_p*(x_i)^p values

    b = fstar.(x); # Values of fstar on range of x

    c = (A'*A) \ (A'*b)  # Vector of coefficients
    
    f(c,x) = sum(c[Integer((p+1)/2)] * x^p for p in 1:2:pmaxodd); # Approximate function
    
    return yy = [f(c,x) for x in xx] 
end

least_squares_odd (generic function with 1 method)

As we did in question 2, we can calculate RMS values using different polynomial orders.

In [23]:
# Searching for the polynomial order (p value) which would yield an rms less than 10e-10 as earlier

pcapodd = 51 # maximum polynomial order
npoints = 1000 # number of points in approximation
x = LinRange(0, π/2, npoints) 
xx = LinRange(0, π/2, 201) 

fstar(x) = sin(x) # target function
ystar = fstar.(xx); # actual values from the target function

rms_vals_odd = zeros(Integer((pcapodd+1)/2)) # record the rms values

for p in 1:2:pcapodd
    #i = floor(p/2) #index
    yy = least_squares_odd(p, npoints, x, fstar, xx) # calculate the approximated y values using the function defined in Question 1
    rms_vals_odd[Integer((p+1)/2)] =  errorl2(yy, ystar);
end

In [25]:
# Plot rms vs polynomial order
fig = Figure()
Axis(fig[1,1], xlabel="Order of maximum polynomial", ylabel=L"$\log_{10}(\text{RMS})$", title = "Root mean square vs maximum odd-polynomial order (logarithmic) " )
scatter!(1:2:pcapodd, log10.(rms_vals_odd)) # approximated values
fig

From the figure above, we can deduce that lower RMS values can be achieved using only the odd polynomials. Again, we see that increasing the order after some value does not improve the error in the code due to numerical errors.

We can compare the computation costs of least_squares_odd(...) and least_squares(...) functions measuring the executionusing time.

In [26]:
using BenchmarkTools # this allows us to use a time function

In [27]:
# Setting up the parameters to use in the functions.
pmax = 20
npoints = 1000
fstar(x) = sin(x)
x = LinRange(0, π*4, npoints)
xx = LinRange(0, π*4, 201);

In [28]:
# Measure execution times
@btime y = least_squares(pmax, npoints, x, fstar, xx); # timing the usual least squares function

  433.288 μs (17 allocations: 182.03 KiB)


In [15]:
@btime y_odd = least_squares_odd(2*pmax+1, npoints, x, fstar, xx); # timing least squares method using only odd polynomials

  502.754 μs (19 allocations: 182.11 KiB)


Note that, in order to run the the functions over the same number of polynomial orders, we evaluate `least_squares` up to $p_{max}$ order; whereas we run `least_squares_odd` up to the order $2p_{max}+1$ (we choose an even $p_{max}$ to start with). 

Comparing the execution times, one can say that the full `least_squares` function works faster than `least_squares_odd` when evaluated over the same number of terms.

## Question 4

4.a. Calculate (manually, not numerically) the derivative of the approximating function $f(c, x) = \sum_p c_p x^p$. Define a new function $g(d, x) = \sum_p d_p x^p$ which depends on a different set of coefficients $|d>$. Set $g(d, x) = f'(c, x)$, and solve for the coefficients $|d>$ as a function of the coefficients $|c>$. Since the derivative is a linear operation, the relation between $|d>$ and $|c>$ can be expressed via a linear operator, the derivative matrix D: $|d> = D|c>$. Calculate D.

We take the derivative of $f(c,x) $with respect to $x$:
$f'(c,x) = \sum_{p} p c_p x^{(p-1)}$. We can shift the series by redefining $\tilde{p}=p-1$
 
$f'(c,x) = \sum_{\tilde{p}=-1} (\tilde{p}+1) c_{\tilde{p}+1} x^{\tilde{p}}$. Furthermore, We can start the series from $\tilde{p}=0$ since the first  term ($\tilde{p}=-1$) is already evaluates to 0. So, dropping the tilde, we have: 

\begin{equation}
f'(c,x) = \sum_{p} (p+1) c_{p+1} x^{p}
\end{equation}

We set this derivative equal to a new function $f'(c,x) \equiv g(d, x) = \sum_{p} d_p x^p$. We can relate the expansion coefficients $d_p$ to coefficients $c_p$ by identifying the two series and matching the coefficients of same orders of $x^p$. That is,

\begin{equation}
\sum_{p} (p+1) c_{p+1} x^{p} = \sum_{p} d_p x^p \rightarrow d_p = (p+1)c_{p+1} 
\end{equation}

We can represent this result using vectors
$|d\rangle = D|c\rangle$.
We can write this linear algebraic equation as:


\begin{gather}
    \begin{bmatrix}
    d_0 \\
    d_1 \\
    \cdot \\
    \cdot \\
    \cdot \\
    d_P
    \end{bmatrix}
    = D
    \begin{bmatrix}
    c_0 \\
    c_1 \\
    \cdot \\
    \cdot \\
    \cdot \\
    c_P
    \end{bmatrix}
\end{gather}

Since we have identified the relation between $d_p$ and $c_p$ values we can determine the matrix $D$. Note that both $|d\rangle $ and $|c\rangle$ are vectors of length $P+1$ since the indices of the components run from 0 to $P$. However, after taking the derivative, we identified $d_p = (p+1)c_{p+1}$. Therefore, we set $d_P = 0$ since the highest order polyomial is not accessible after differentiation. Note also that, $D$ has to be a $(P+1)\times(P+1)$ matrix.

Using the relation $d_p = (p+1)c_{p+1}$, we can write:

\begin{gather}
    \begin{bmatrix}
    c_1 \\
    2c_2 \\
    \cdot \\
    \cdot \\
    \cdot \\
    Pc_P \\
    0
    \end{bmatrix}
    = 
    \begin{bmatrix}
    0 & 1 & 0 & 0 & \cdot & \cdot & \cdot & 0 \\
    0 & 0 & 2 & 0 & \cdot & \cdot & \cdot & 0 \\
    \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot\\
    0 & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & {P} \\
     0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
    \end{bmatrix}
    \begin{bmatrix}
    c_0 \\
    c_1 \\
    \cdot \\
    \cdot \\
    \cdot \\
    c_{P-1} \\
    c_P
    \end{bmatrix}
\end{gather}

Finally, we identify the matrix $D$ as the $(P+1)\times(P+1)$ matrix:
\begin{gather}
    D
    = 
    \begin{bmatrix}
    0 & 1 & 0 & 0 & \cdot & \cdot & \cdot & 0 \\
    0 & 0 & 2 & 0 & \cdot & \cdot & \cdot & 0 \\
    \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot\\
    0 & \cdot & \cdot & 0 & p+1 & 0 & \cdot & 0 \\
    \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot\\
    \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & \cdot & {P} \\
     0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
    \end{bmatrix}
\end{gather}


4.b. Calculate D numerically, and test your code by comparing (1) approximating sin(x) and calculating the derivative via D, and (2) approximating sin'(x) = cos(x). (Note that cos(x) cannot be approximated well by antisymmetric polynomials, i.e. don't use your code from task 3 above.)

In [30]:
# Set the initial parameters as earlier.
pmax = 10;
npoints = 10;
x = LinRange(0, π/2, npoints)
xx = LinRange(0, π/2, 201) 
fstar(x) = sin(x)

# Calculate coefficients c
A = [x[i]^p for i in 1:npoints, p in 0:pmax] # Matrix of c_p*(x_i)^p values
b = fstar.(x); # Values of fstar on range of x
c = (A'*A) \ (A'*b)  # Vector of coefficients

# Create the differentiation matrix D as prescribed in 4.a
D = zeros(pmax+1,pmax+1)
for i in 1:pmax
    D[i,i+1] = i
end

Now, we directly calculate the coefficients vector $|d\rangle$ via applying our approximation scheme on $\sin'{(x)} = \cos{x} $ and compare this against the calculation of derivative using the matrix $D$.

In [31]:
# Approximate cosine with the same expansion scheme.
#Set the initial parameters as earlier.
pmax = 10;
npoints = 10;
x = LinRange(0, π/2, npoints)
xx = LinRange(0, π/2, 201) 
fstar′(x) = cos(x) # use cosine this time

# Calculate coefficients c
A′ = [x[i]^p for i in 1:npoints, p in 0:pmax] # Matrix of c_p*(x_i)^p values
b′ = fstar′.(x); # Values of fstar on range of x
d_direct = (A′'*A′) \ (A′'*b′)  # Vector of coefficients

11-element Vector{Float64}:
  0.9999999999997035
  1.0152934092135611e-5
 -0.5001645342430868
  0.001076639752816978
  0.037860932508914785
  0.008112842660085025
 -0.012306286088970018
  0.009339808386193877
 -0.004900475525972112
  0.0014592259838960221
 -0.0001860048202425842

In [32]:
# Plot the approximated function after differentiation and compare with cosine
f(c,x) = sum(c[p+1] * x^p for p in 0:pmax) # Approximate function

fig = Figure()
ax = Axis(fig[1,1], xlabel=L"x", ylabel=L"f(x)", title = L"Derivative of $f*(x)$" )
lines!(xx, [f(d_direct,x) for x in xx], color=:red, linestyle=:solid, label="least squares") # cos approximated via least squares
scatter!(xx, [f(D*c,x) for x in xx], markersize=3, color=:blue, label="derivative matrix") # cos calculated via acting D
fig[1,2] = Legend(fig, ax)
fig

From the figure above, we see that using the linear derivative operator $D$ and approximating the function via least squares agree well on the interval we implemented the calculation.